In [450]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [451]:
crime=pd.read_csv(r'C:\Users\Rachel\Desktop\ML\final\crime19.csv')

In [452]:
print(crime.shape)

(2115055, 28)


In [453]:
crime.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA ', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

In [454]:
labels=crime[['AREA ', 'AREA NAME', 'Crm Cd','Crm Cd Desc', 'Premis Cd','Premis Desc','Weapon Used Cd', 'Weapon Desc','Status','Status Desc']]

In [455]:
crime=crime.drop(['DATE OCC', 'AREA NAME',
       'Crm Cd Desc','TIME OCC' ,'Mocodes',
        'Vict Descent',  'Premis Desc',
        'Weapon Desc',  'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'Cross Street', 'LAT','LON'], axis=1)

In [456]:
crime=crime.rename(columns={'DR_NO':'id', 'Date Rptd':'date', 'AREA ':'area', 'Rpt Dist No':'district_number', 'Part 1-2':'part', 'Crm Cd':'crime_code', 
       'Vict Age':'age', 'Vict Sex':'sex', 'Vict Descent':'descent', 'Premis Cd':'premis', 
       'Weapon Used Cd':'weapon', 'Status':'status', 'LOCATION':'location'})

In [457]:
token='N'
crime.sex=crime.sex.fillna(token)
crime.weapon=crime.weapon.fillna(token)

crime=crime.dropna(how='any')
print(crime.isnull().sum())

id                 0
date               0
area               0
district_number    0
part               0
crime_code         0
age                0
sex                0
premis             0
weapon             0
status             0
location           0
dtype: int64


In [458]:
crime['month_year'] =pd.to_datetime(crime['date']).dt.to_period('M')

In [459]:
crime=crime.drop(['date'], axis=1)

In [460]:
# there are many data which age = 0 doesn't make any  sense
crime[crime.age==0].shape

(369863, 12)

In [461]:
crime_11_12=crime[crime.month_year>'2011-11'].sort_values(by='month_year').reset_index(drop=True)

In [462]:
crime_11_12.shape

(1732875, 12)

In [463]:
print(crime_11_12['month_year'].unique())

<PeriodArray>
['2011-12', '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
 '2012-07', '2012-08', '2012-09',
 ...
 '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05',
 '2020-06', '2020-07', '2020-08']
Length: 105, dtype: period[M]


## spending

In [464]:
# spending=pd.read_csv(r'C:\Users\Rachel\Desktop\ML\final\spending.csv')
# spending.shape
# pd.set_option('display.max_columns', None)
# # spending.iloc[:,:30].isnull().sum()
# # spending.iloc[:,30:].isnull().sum()
# labels=spending(['ITEM CODE NAME','ITEM CODE'])
# spending.columns
# spending.info()
# spending.isnull().sum()
# spending=spending.drop(['SETTLEMENT/JUDGMENT','AUTHORITY LINK','AUTHORITY','AUTHORITY NAME','INVOICE DISCOUNT DUE DATE','UNIT OF MEASURE','INVOICE DUE DATE','BU NAME','CURRENCY'], axis=1)

### Budegt

In [465]:
budget=pd.read_csv(r'C:\Users\Rachel\Desktop\ML\final\budget.csv')

In [466]:
#drop columns after total expenditures
budget=budget.iloc[:,:32]

In [467]:
# budget.columns

In [468]:
#drop same values columns
#'DEPARTMENT', 'DEPARTMENT NAME','AFR TYPE','DEPARTMENTAL INDICATOR','FUND GROUP','FUND GROUP NAME'bu
budget=budget.drop(['FUND GROUP NAME','FUND NAME','FISCAL PERIOD NAME','FUND TYPE NAME','DEPARTMENT', 'DEPARTMENT NAME','ACCOUNT NAME', 'CAFR TYPE','ACCOUNT TYPE NAME','DEPARTMENTAL INDICATOR','ACCOUNT GROUP NAME','FUND GROUP','FUND GROUP NAME'], axis=1)

In [469]:
#drop missing values morethan 20000.
budget=budget.drop(['ADOPTED EXPENSE BUDGET','REVENUE SOURCE CATEGORY','REVENUE SOURCE CATEGORY NAME','REVENUE SOURCE CLASS','REVENUE SOURCE CLASS NAME','REVENUE SOURCE','REVENUE SOURCE NAME'], axis=1)

In [470]:
token='UNK'
budget=budget.fillna(token)

In [471]:
a=budget.columns.map(lambda x: x.lower().replace(' ', '_'))
budget.columns=a

In [472]:
budget['month_year']=pd.to_datetime(budget['fiscal_period'], format='%Y%m').dt.to_period('M')

In [473]:
budget=budget.drop(['fiscal_period'], axis=1)

In [474]:
budget.columns

Index(['budget_fiscal_year', 'fiscal_year', 'fund_type', 'fund',
       'account_group', 'account_type', 'account', 'activity', 'activity_name',
       'activity_group', 'activity_group_name', 'total_expenditures',
       'month_year'],
      dtype='object')

In [475]:
budget=budget.sort_values(by='month_year').reset_index(drop=True)

In [476]:
total=crime_11_12.groupby(['month_year']).size().to_frame()

In [477]:
crime_11_12=pd.merge(crime_11_12, total, on='month_year',how='left')

In [478]:
crime_11_12.rename(columns={0:'crime'})

,id,area,district_number,part,crime_code,age,sex,premis,weapon,status,location,month_year,crime
0,111721860,17,1756,1,330,32,M,104.0,N,IC,17600 MAYALL ST,2011-12,16246
1,111522020,15,1527,1,442,0,M,404.0,N,IC,11000 VICTORY BL,2011-12,16246
2,111432710,14,1483,1,510,0,N,101.0,N,IC,LINCOLN,2011-12,16246
3,111924380,19,1987,2,354,55,M,501.0,N,IC,8600 KATHERINE AV,2011-12,16246
4,110120970,1,191,1,510,0,N,101.0,N,IC,17TH ST,2011-12,16246
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732870,201711824,17,1761,1,331,49,M,509.0,N,IC,21500 LASSEN ST,2020-08,8
1732871,201912858,19,1969,1,121,26,F,502.0,400,IC,ARLETA,2020-08,8
1732872,200115750,1,121,2,354,36,F,710.0,N,IC,900 W TEMPLE ST,2020-08,8
1732873,202012875,20,2029,2,649,42,M,502.0,N,IC,500 S WESTMORELAND AV,2020-08,8


In [479]:
# df=pd.merge(crime_11_12, budget, on ="month_year", how = "outer")

In [480]:
budget=pd.merge(budget, total,  on='month_year',how='left')
budget.rename(columns={0:'crime'})

,budget_fiscal_year,fiscal_year,fund_type,fund,account_group,account_type,account,activity,activity_name,activity_group,activity_group_name,total_expenditures,month_year,crime
0,2008,2011,R,44D,UNK,2,70C436,9999,PROGRAM UNKNOWN,9,TEMPORARY ROLL UP,23389.03,2011-12,16246.0
1,2012,2011,R,554,UNK,2,70H110,UNK,NOT ENTERED,UNK,NOT ENTERED,0.00,2011-12,16246.0
2,2005,2011,R,44D,UNK,2,70W510,UNK,NOT ENTERED,UNK,NOT ENTERED,0.00,2011-12,16246.0
3,2011,2011,R,44E,UNK,2,70G514,UNK,NOT ENTERED,UNK,NOT ENTERED,0.00,2011-12,16246.0
4,2011,2011,R,339,UNK,2,70G499,UNK,NOT ENTERED,UNK,NOT ENTERED,0.00,2011-12,16246.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31831,2020,2020,G,100,SAL,1,1090,7004,CUSTODY OF PERSONS AND PROPERTY,A,COMMUNITY SAFETY,25164.98,2020-10,NaN
31832,2020,2020,R,44D,UNK,UNK,UNK,UNK,NOT ENTERED,UNK,NOT ENTERED,0.00,2020-10,NaN
31833,2020,2020,G,100,EXP,1,4430,7048,OPERATIONAL SUPPORT,A,COMMUNITY SAFETY,108939.96,2020-10,NaN
31834,2020,2020,G,100,SAL,1,1092,9999,PROGRAM UNKNOWN,9,TEMPORARY ROLL UP,504.30,2020-10,NaN
